#Imports

In [1]:
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Install sentence transformer

In [2]:
#!pip install -U sentence-transformers

# **Preprocessing using Camel Tools**

In [3]:
# !pip install camel-tools -f https://download.pytorch.org/whl/torch_stable.html


In [4]:
# !pip install camel-tools -f 
# !camel_data -i all

In [5]:
# -*- coding: utf-8 -*- #to be able to process arabic
import pandas as pd
import re
from camel_tools.utils.dediac import dediac_ar
from camel_tools.utils.charsets import AR_LETTERS_CHARSET
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.disambig.mle import MLEDisambiguator
mle_pretrained = MLEDisambiguator.pretrained()

In [6]:
stopwords = []
with open('F:/Gp/Question Answering/arabic-stop-words-master/list.txt', encoding='utf8') as file:
    for line in file:
        stopwords.append(line.strip())
print(stopwords)
print(type(stopwords))  

['،', 'ء', 'ءَ', 'آ', 'آب', 'آذار', 'آض', 'آل', 'آمينَ', 'آناء', 'آنفا', 'آه', 'آهاً', 'آهٍ', 'آهِ', 'أ', 'أبدا', 'أبريل', 'أبو', 'أبٌ', 'أجل', 'أجمع', 'أحد', 'أخبر', 'أخذ', 'أخو', 'أخٌ', 'أربع', 'أربعاء', 'أربعة', 'أربعمئة', 'أربعمائة', 'أرى', 'أسكن', 'أصبح', 'أصلا', 'أضحى', 'أطعم', 'أعطى', 'أعلم', 'أغسطس', 'أفريل', 'أفعل به', 'أفٍّ', 'أقبل', 'أكتوبر', 'أل', 'ألا', 'ألف', 'ألفى', 'أم', 'أما', 'أمام', 'أمامك', 'أمامكَ', 'أمد', 'أمس', 'أمسى', 'أمّا', 'أن', 'أنا', 'أنبأ', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'أنتِ', 'أنشأ', 'أنه', 'أنًّ', 'أنّى', 'أهلا', 'أو', 'أوت', 'أوشك', 'أول', 'أولئك', 'أولاء', 'أولالك', 'أوّهْ', 'أى', 'أي', 'أيا', 'أيار', 'أيضا', 'أيلول', 'أين', 'أيّ', 'أيّان', 'أُفٍّ', 'ؤ', 'إحدى', 'إذ', 'إذا', 'إذاً', 'إذما', 'إذن', 'إزاء', 'إلى', 'إلي', 'إليكم', 'إليكما', 'إليكنّ', 'إليكَ', 'إلَيْكَ', 'إلّا', 'إمّا', 'إن', 'إنَّ', 'إى', 'إياك', 'إياكم', 'إياكما', 'إياكن', 'إيانا', 'إياه', 'إياها', 'إياهم', 'إياهما', 'إياهن', 'إياي', 'إيهٍ', 'ئ', 'ا', 'ا?', 'ا?ى', 'االا', 'االتى', 'اب

In [7]:
def clean_text(text):
    sentence_ar_dediac = dediac_ar(text) #dediacritization using camel tools
    sentence_ar_dediac = re.sub(r"\S*https?:\S*|@\S+", "", sentence_ar_dediac) #remove URLs and mentions even if they are in paranthesis or brackets
    no_punc = ""
    for char in sentence_ar_dediac:
        if char in (list(AR_LETTERS_CHARSET)+[" "]): #removing any thing that is not an arabic letter
            no_punc = no_punc + char
    return no_punc

In [8]:
def normalize_text(text):
    cleaned_text = normalize_alef_maksura_ar(text)
    cleaned_text = normalize_alef_ar(cleaned_text)
    cleaned_text = normalize_teh_marbuta_ar(cleaned_text)
    cleaned_text = normalize_unicode(cleaned_text)
    
    return cleaned_text

In [9]:
def enrichement(text, stopwords=stopwords, mle_pretrained=mle_pretrained):
    tokens = simple_word_tokenize(text) #tokenize the sentence
    tokenized = []
    for token in tokens: #remove stop words
        if token not in stopwords:
            tokenized.append(token)     
    disambig = mle_pretrained.disambiguate(tokenized) #return each word to its base form
    lemmas = [d.analyses[0].analysis['lex'] for d in disambig]
    whole_sent = " ".join(lemmas) # join to get the whole sentence
    whole_sent_dediac = dediac_ar(whole_sent) #diacritize the sentence 
    return whole_sent_dediac

In [10]:
def preprocess_text(text, clean_text=clean_text, normalize_text=normalize_text, enrichement=enrichement):
    '''
    Inputs:
    text: string in which it will preprocess.
    
    Outputs:
    cleaned_text: the string after it was processed.
    '''
    
    #let's first start with cleaning 
    cleaned = clean_text(text)
            
    # now, for the normalization part
    normalized = normalize_text(cleaned)
    
    #enrichement
    enriched = enrichement(normalized)
    
    return enriched

In [11]:
# def preprocess_data(data, preprocess_text=preprocess_text):
#     '''
#     Inputs:
#     data: is the dataframe we want to apply our function on
    
#     Outputs:
#     data: the dataframe after applying the cleaning and normalization on each cell.
    
#     '''

#     for row in range(data.shape[0]):
       
#             for_cleaning = str(data.iloc[row][0])
#             data.iloc[row][0] = preprocess_text(for_cleaning)
#     return data
     

In [12]:

# data_prep=[]
# for row in range(data.shape[0]):
       
#         #for_cleaning = str(data.iloc[row][3])
#         data_prep.append(preprocess_text(data.iloc[row][3]))
   

#The Model

In [13]:
from sentence_transformers import SentenceTransformer
model_name="aubmindlab/bert-base-arabertv02"

In [14]:
from torch.utils.data import DataLoader
from torch import nn
from sentence_transformers import LoggingHandler, SentenceTransformer, util, models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
import pandas as pd

In [15]:
batch_size = 64
max_seq_length=300
word_embedding_model = models.Transformer(model_name, max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True)
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), 
                           out_features=256, 
                           activation_function=nn.Tanh())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
def Classification(data,classification):
  classQuestions=[]
  classAnswers=[]
  classEmbeddings=[]
  for i in range(len(data)):
    if data['التصنيف'][i]==classification:
      classAnswers.append(data['الاجابات'][i])
      classQuestions.append(data['الاسئله'][i])
  classEmbeddings=model.encode(classQuestions) 
  return classQuestions,classAnswers, classEmbeddings
  # print(classAnswers)
  # len(classQuestions)


In [32]:
type(classEmbeddings[0][0])

str

##similarity and evaluation

In [40]:
def get_answer(q_encode,classEmbeddings,classAnswers):
  x_val = cosine_similarity(
      [q_encode[0]],
      classEmbeddings[:]
  )
  percentage=max(x_val[0])#percentage of the similarity
  ANSWER=x_val.argmax() #get answer index
  return classAnswers[ANSWER]

In [41]:
data=pd.read_csv("C:/Users/Dalia/Downloads/Fatawy_final_version.csv")
data

,Unnamed: 0,الاسئله,الاجابات,التصنيف
0,0,قال سائل علم ليل قدر خير شهر حرص إسلام حث مسلم...,إن الله أخفى رضاه في طاعته؛ ليستزيد أصحاب الطا...,عبادات و الذكر
1,1,قال سائل حكم رخصة إفطار مسافر رمضان ذلك ٱستكمل...,إن الله سبحانه وتعالى رخَّص للمسافر في رمضان أ...,المسافر
2,2,جاز إفطار حامل مرض خاف صيام جنين ربيع هل لزم ق...,يجوز للحامل والمرضع الإفطار إذا خافتا على الجن...,عبادات و الصوم
3,3,أراد سائل أقام حاجرا رخام مقبر أسرة حتى معروف ...,أجاز الأحناف كتابة اسم الميت فقط على القبر عند...,عبادات و الجنائز
4,4,قاعدة نصب شريعة إسلامي تنمية ذبح شرعي حيوان طير,الذكاة الشرعية أي الذبح الشرعي معناه في شريعة ...,آداب وأخلاق
...,...,...,...,...
4508,4508,أمر تم خطبة رجل عمل خارج لما أراد عقد طلب ٱسم ...,هذا العقد باطل؛ لأنه لم يحصل فيه توكيل بعقد ال...,مجتمع وأسرة من النكاح
4509,4509,تقدم طلب يد ٱبن عم فتفاجات أم أبلغ ٱبن عم أرضع...,إذا رضع كل منكما من زوجة العم هذه خمس رضعات مش...,مجتمع وأسرة من الرجاء
4510,4510,شاب أراد زواج فتاة رفعت والد أخت أصغر ركعة مطب...,المقرر شرعًا أنه يحرم من الرضاع ما يحرم من الن...,مجتمع وأسرة من الرجاء
4511,4511,خرج كفارة ني أم مطبوخ,جمهور الفقهاء يشترطون في إخراج الكفارة التمليك...,شؤون عادات و الذبائح في الأضحية في النذور


In [42]:

classification='جنايات وقضية ما جنايات'
# question=[preprocess_text('هتنظيم ديني متطرف أعلن تقديم مكافأة مالي قتل دبلوماسي عسكري ود معرفة حكم شرعي ما أول كرة قتل ثاني قتل شخص دخل مظلة عقد أمان حماية دبلوماسي سائح أجنبي مقيم بلد إسلامي غرض عمل')]
question=['هتنظيم ديني متطرف أعلن تقديم مكافأة مالي قتل دبلوماسي عسكري ود معرفة حكم شرعي ما أول كرة قتل ثاني قتل شخص دخل مظلة عقد أمان حماية دبلوماسي سائح أجنبي مقيم بلد إسلامي غرض عمل']
q_encode=model.encode(question)
classQuestions,classAnswers,classEmbeddings=Classification(data,classification)
answer=get_answer(q_encode,classEmbeddings,classAnswers)
answer

'الاعتداء على النفس الإنسانية بالأذى أو القتل بغير حقٍّ حرامٌ شرعًا ومن أكبر الكبائر؛ سواء كان ذلك واقعًا على الدبلوماسيين أو السائحين أو الأجانب المقيمين في البلاد الإسلامية بغرض العمل فيها أو غيرهم؛ لأن الأصل في نفوسهم ودمائهم وأموالهم أنها معصومة ومحرَّمة، كما أن وجودهم تابع لما أخذوه بعهد الأمان التابع لتأشيرة الدخول، ولا يجوز مخالفة عهود الأمان؛ لما في ذلك من الغدر المحرَّم، ولارتباطها وتعلقها بالذمم؛ قال تعالى: ﴿يَا أَيُّهَا الَّذِينَ آمَنُوا أَوْفُوا بِالْعُقُودِ﴾ [المائدة: 1]، وقال صلى الله عليه وآله وسلم: «مَنْ قَتَلَ مُعَاهَدًا لَمْ يَرِحْ رَائِحَةَ الجَنَّةِ، وَإِنَّ رِيحَهَا تُوجَدُ مِنْ مَسِيرَةِ أَرْبَعِينَ عَامًا» رواه البخاري.أما عن بذل المال للغير نظير قيامه بتلك الاغتيالات؛ فهو من الإجارات الفاسدة التي لا تنعقد، وبذل المال من المحرِّض وقبوله من المحرَّض أمران محرَّمان شرعًا؛ لقوله تعالى: ﴿وَلَا تَعَاوَنُوا عَلَى الْإِثْمِ وَالْعُدْوَانِ﴾ [المائدة: 2]، ولأن المنفعة المقصودة فيه محرَّمة وهي القتل وسفك الدماء، فكان بذل المال في سبيلها محرَّم أيضًا.                       